#### importing libraries

In [1]:
import pandas as pd
import bs4
import selenium
import requests
import urllib
import csv
from sklearn.metrics.pairwise import cosine_similarity

#### reading the zomato url

In [3]:
from selenium import webdriver
browser = webdriver.Chrome('chromedriver.exe')
url1 = 'https://www.zomato.com/bangalore/great-food-no-bull'
browser.get(url1)

In [6]:
urls = browser.find_elements_by_css_selector('div.relative.top-res-box.entity-ads-snippet-track')

#### 	From this URL (https://www.zomato.com/bangalore/great-food-no-bull), scrape the following information and store them in a data frame
- Restaurant Name
- URL to the restaurant


In [7]:
urls_df = pd.DataFrame(columns=['Restaurant_Name', 'URL'])
for elem in urls:
    url = elem.find_element_by_css_selector('a')
    url = url.get_attribute('href')
    rest_name = elem.find_element_by_css_selector('div.res_title')
    rest_name = rest_name.get_attribute('innerHTML')
    curr_row = {
        'URL' : url, 
        'Restaurant_Name' : rest_name
    }
    
    urls_df = urls_df.append(curr_row,ignore_index = True)
    urls_df['Restaurant_Name'] = urls_df['Restaurant_Name'].str.replace('\n','')
    row = [url,rest_name.strip()]
    with open('rest.csv', 'a', newline = '') as csvFile: ### writing the dataframe to a csv file
        writer = csv.writer(csvFile)
        writer.writerow(row)

In [8]:
urls_df

,Restaurant_Name,URL
0,Vapour Brewpub and Diner,https://www.zomato.com/bangalore/vapour-brewpu...
1,The Old Fashioned Bar,https://www.zomato.com/bangalore/the-old-fashi...
2,Mahjong Room - WelcomHotel... ...,https://www.zomato.com/bangalore/mahjong-room-...
3,Burma Burma,https://www.zomato.com/bangalore/burma-burma-i...
4,Lush - Renaissance Hotel...,https://www.zomato.com/bangalore/lush-renaissa...
5,Suryawanshi,https://www.zomato.com/bangalore/suryawanshi-s...
6,The Hole in the Wall Cafe,https://www.zomato.com/bangalore/the-hole-in-t...
7,Apsara Ice Cream,https://www.zomato.com/bangalore/restaurants/a...
8,Lavonne,https://www.zomato.com/bangalore/lavonne-indir...
9,Once Upon A Flame,https://www.zomato.com/bangalore/once-upon-a-f...


####  Loop through each restaurant(atleast 50) and do the following
- scrape sample reviews, make sure you click the load more button continuously in a loop until all reviews are loaded
- From each review extract the following
    - Reviewer name
    - Reviewer ID
    - Ratings


In [ ]:
reviews_df = pd.DataFrame(columns=['Restaurant_name','User_ID','User_Name','Rating']) #,'Reviews'
for a,elem1 in urls_df.iterrows():
    try: ## to catch the exceptions that we get while scraping data from restaurants(to skp these kind of restaurantsS)
        browser.get(elem1['URL'])
        rest_name = elem1['Restaurant_Name']
        while(True): ### load more
            try:
                more_sel = 'div.load-more'
                load_more_button = browser.find_element_by_css_selector(more_sel)
                load_more_button.click()
            except Exception:
                break;
        reviews = browser.find_elements_by_css_selector('div.ui.segment.clearfix.brtop')
        for elem in reviews:
            name = elem.find_element_by_css_selector('a[data-entity_id]')
            uid = name.get_attribute('data-entity_id')
            name = name.get_attribute('innerHTML')#.splitlines()[1]
            rating =  elem.find_element_by_css_selector('div[aria-label]')
            rating = rating.get_attribute('aria-label')
            curr_row = {
                'Restaurant_name' : rest_name,
                'User_ID' : uid,
                'User_Name': name,
                'Rating' : rating,
                #'Reviews' : rev
            }

            reviews_df = reviews_df.append(curr_row,ignore_index = True)
            reviews_df['Rating'] = reviews_df['Rating'].str.replace('[^0-9.]','')
            reviews_df['User_Name'] = reviews_df['User_Name'].str.replace('[^a-zA-Z]','')
            #reviews_df['Reviews'] = reviews_df['Reviews'].str.replace('[^a-zA-Z0-9 ]','')
            #reviews_df['Reviews'] = reviews_df['Reviews'].str.replace('RATED','')
            row = [rest_name.strip(), uid,name.strip(),rating]
            with open('review.csv', 'a', newline='') as csvFile: ### writing the dataframe to a csv file
                writer = csv.writer(csvFile)
                writer.writerow(row)
    except Exception:
                continue
    if(str(a)=='70'): ### restricting the data to 70 rows so that we get details of atleast 50 restaurants
        break

#### writing the data from dataframe to a csv file

In [50]:
reviews = pd.read_csv('review.csv', encoding = "latin")
reviews.head()

,Restaurant_Name,User_ID,User_Name,Rating
0,1131 Bar + Kitchen,1071516,Aritra,4.0
1,1131 Bar + Kitchen,31647122,Priyanka Gupta,4.0
2,1131 Bar + Kitchen,47274603,Foodiebaay,3.0
3,1131 Bar + Kitchen,6650341,Biswajit,5.0
4,1131 Bar + Kitchen,3200860,Kuldeep Behera,4.0


#### pivot table for the reviews DF

In [51]:
user_item_matrix = reviews.pivot_table(index='User_ID',
                   columns = 'Restaurant_Name',
                   values='Rating')
print(user_item_matrix.shape)
user_item_matrix.head()

(1608, 56)


Restaurant_Name,1131 Bar + Kitchen,"3B's - Buddies, Bar &amp;...",99 Pancakes,Arbor Brewing Company,Arogya Ahaara,Avista Cafe Pizzeria,BITEBOX,BOX8- Desi Meals,Bamboo Hut,Big Pitcher,...,The Asian Curry House,The Globe Grub,The Permit Room,The Pizza Bakery,The Terrace at Gilly's...,Toast &amp; Tonic,Veena Stores,Yauatcha,Zamindar,nu.tree
User_ID,,,,,,,,,,,,,,,,,,,,,
7693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18535,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Perform exploratory data analysis using the above data

#### 1. Avg no. of ratings per user

In [52]:
user_item_matrix.count(axis=1).mean() ## per user avg

1.2898009950248757

#### 2. User with most no. of ratings

In [53]:
user_item_matrix.count(axis=1).sort_values(ascending = False).head(1)

User_ID
1294851    5
dtype: int64

#### 3. Avg rating per restaurant

In [54]:
user_item_matrix.count().mean()

37.035714285714285

#### 4. Restaurant with most no. of ratings

In [55]:
user_item_matrix.count().sort_values(ascending = False).head(1)

Restaurant_Name
Communiti    145
dtype: int64

##### cosine_similarity does not accept missing values

In [56]:
ui_matrix = user_item_matrix.apply(lambda v: v - v.mean(), axis = 1).fillna(0)

In [57]:
sim_matrix = pd.DataFrame(cosine_similarity(ui_matrix),
            index = ui_matrix.index,
            columns=ui_matrix.index)

In [58]:
sim_matrix

User_ID,7693,14340,18535,19800,21472,23288,25223,26044,32856,35823,...,71778704,72508033,72885873,73637155,74112908,76768454,77841871,78195007,82913026,83298879
User_ID,,,,,,,,,,,,,,,,,,,,,
7693,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 5.	User the above data, build a recommendation engine using User Based Collaborative Filtering. Create a function which takes User ID as input and return 3 restaurants that you would recommend to the user.

In [59]:
def ubcf(userid): 
    preds = pd.DataFrame(columns=['rname', 'predicted_rating'])
    neighs_user = sim_matrix[userid].drop(userid).sort_values(ascending = False).head(7).index
    for rname, rating in user_item_matrix.loc[userid].items(): 
        if pd.isnull(rating):
            rname_ratings = user_item_matrix.loc[neighs_user, rname]
            avg_rname_rating = rname_ratings.mean() # predicting the rating for the restaurants based on the avg rating given by neighbours
            preds = preds.append({'rname' : rname,
                                 'predicted_rating': avg_rname_rating},
                                 ignore_index = True)
            rest_recom = preds.sort_values('predicted_rating', ascending=False).head(3)['rname']
    return rest_recom.values.tolist()

#### Sample input and output. 

In [60]:
ubcf(19800)

['1131 Bar + Kitchen', 'Deja Vu Resto Bar', 'Punjab Grill']